In [12]:
DATA = "/content/drive/MyDrive/IIIT/indicvoices_rsml_ready.csv"

In [13]:
!pip install jiwer

In [14]:
import pandas as pd

df = pd.read_csv(DATA)
print("Columns in the CSV file:")
for col in df.columns:
    print(col)

/tmp/ipython-input-187391771.py:3: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA)


Columns in the CSV file:
text
duration
lang
samples
verbatim
normalized
speaker_id
scenario
task_name
gender
age_group
job_type
qualification
area
district
state
occupation
verification_report
unsanitized_verbatim
unsanitized_normalized
file
segment
audio
batch
rsml


In [15]:
# Disfluency tags
DISFLUENCY_TAGS = [
    "tsk",
    "stammers",
    "ugh",
    "uhh",
    "hmm",
    "umm",
    "uh-huh",
    "sigh"
]

# Noise tags
NOISE_TAGS = [
    # Human / baby / child sounds
    "baby",
    "baby_crying",
    "baby_talking",
    "child",
    "child_crying",
    "child_laughing",
    "child_talking",
    "child_yelling",
    "child_whining",
    "children",
    "children_talking",
    "children_yelling",
    "laughter",
    "talking",
    "yelling",
    "whispering",
    "singing",
    "whistling",
    "gasp",
    "groan",

    # Breathing / physiological
    "breathing",
    "inhaling",
    "sniffle",
    "sniffing",
    "nose_blowing",
    "cough",
    "sneezing",
    "throat_clearing",
    "yawning",
    "swallowing",
    "wheezing",
    "snorting",
    "smack",

    # Animal sounds
    "animal",
    "barking",
    "meow",
    "bird_squawk",
    "squawking",

    # Mechanical / object sounds
    "click",
    "clicking",
    "clink",
    "clinking",
    "dishes",
    "door",
    "footsteps",
    "printer",
    "typewriter",
    "scratching",
    "tapping",
    "popping",
    "pounding",
    "thumping",
    "rattling",
    "rustling",
    "screeching",
    "squeak",
    "clanking",
    "clanging",
    "tones",
    "tone",
    "trill",

    # Vehicles / alarms
    "horn",
    "motorcycle",
    "siren",

    # Electronic / signal sounds
    "beep",
    "bell",
    "buzz",
    "buzzer",
    "ringing",
    "phone_ringing",
    "phone_vibrating",
    "chiming",
    "static",
    "hiss",

    # Background / generic noise
    "music",
    "TV",
    "noise",
    "persistent-noise-start",
    "persistent-noise-end",
    "unintelligible"
]


##Code mixed


## Code-Mixing Density (CMD)

$$
\rho_c = \frac{c}{N}
$$

Where:
- $\rho_c$ = Code-Mixing Density  
- c   = Number of code-mixed words  
- N   = Total number of words in the utterance  

Code-Mixing Density is defined as the frequency of all code-mixed words
divided by the total number of words in a given utterance.


In [17]:
import numpy as np
import pandas as pd
import re

# Ensure DATA and df are defined if the cell is run independently
DATA = "/content/drive/MyDrive/IIIT/indicvoices_rsml_ready.csv"
df = pd.read_csv(DATA)

# Function to count all "meaningful" words in unsanitized_normalized for the denominator
def count_meaningful_words_for_density(text):
    if pd.isna(text) or not isinstance(text, str):
        return 0
    words = str(text).split()
    meaningful_words_count = 0
    for word in words:
        # Ignore words starting with '@' (e.g., @noise-start)
        if word.startswith('@'):
            continue
        # If it's a bracketed word, check if it's a known disfluency/noise tag
        if word.startswith('[') and word.endswith(']') and word[1:-1].isalpha():
            content = word[1:-1].lower()
            if content in [tag.lower() for tag in DISFLUENCY_TAGS] or content in [tag.lower() for tag in NOISE_TAGS]:
                # Exclude noise/disfluency tags from total count if they are not considered 'meaningful' content
                continue
            else:
                # Count code-mixed words (e.g., [english]) as a single word in the total
                meaningful_words_count += 1
        else:
            # Count regular words
            meaningful_words_count += 1
    return meaningful_words_count

# Function to count code-mixed words (words in brackets that are not disfluency or noise tags)
def count_code_mixed_words_specific(text):
    if pd.isna(text) or not isinstance(text, str):
        return 0
    words = str(text).split()
    code_mixed_count = 0
    for word in words:
        # Check if the word is in brackets and its content is alphabetic and not a known tag
        if word.startswith('[') and word.endswith(']') and word[1:-1].isalpha() and word[1:-1].lower() not in [tag.lower() for tag in DISFLUENCY_TAGS] and word[1:-1].lower() not in [tag.lower() for tag in NOISE_TAGS]:
            code_mixed_count += 1
    return code_mixed_count

# Calculate total meaningful words from 'unsanitized_normalized' for the denominator
df['total_meaningful_words'] = df['unsanitized_normalized'].apply(count_meaningful_words_for_density)

# Count code-mixed words in 'unsanitized_normalized' (numerator)
df['code_mixed_words_count'] = df['unsanitized_normalized'].apply(count_code_mixed_words_specific)

# Calculate code-mix density, handling division by zero
df['code_mix_density'] = np.where(
    df['total_meaningful_words'] > 0,
    df['code_mixed_words_count'] / df['total_meaningful_words'],
    0
)

print("DataFrame with new 'code_mix_density' column calculated and added.")

# Drop the temporary columns used for density calculation
df = df.drop(columns=["total_meaningful_words", "code_mixed_words_count"], errors='ignore')

# Display head of df with the new 'code_mix_density' column to confirm
display(df[['unsanitized_normalized', 'code_mix_density']].head())

/tmp/ipython-input-1434239888.py:7: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA)


DataFrame with new 'code_mix_density' column calculated and added.


,unsanitized_normalized,code_mix_density
0,@noise-start ఈ వార్తా ధార నుండి తాజా ముఖ్యాంశా...,0.000000
1,నేను నాలుగు ఎనర్జాయర్స్ [energiors] ఆల్కలీన్ [...,0.266667
2,@noise-start ఆమె గతంలో పంతొమ్మిది వందల తొంభై ఏ...,0.214286
3,ఇంపాల్‌లో వాతావరణం ఎలా ఉంది,0.000000
4,@noise-start సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ...,0.142857


## Disfluency Density

## Disfluency Density

Disfluency Density is defined as the frequency of all disfluency tags divided by
the total number of words in a given utterance.

$$
\rho_d = \frac{d}{N}
$$

Where:
- $\rho_d$ = Disfluency Density
- d = Number of disfluency tags in the utterance
- N = Total number of words in the utterance, including disfluencies


In [36]:
import re
import pandas as pd
import numpy as np

# Function to calculate disfluency density based on the user's new logic
def calculate_disfluency_density_user_logic(text):
    if pd.isna(text) or not isinstance(text, str):
        return 0.0

    # Remove all <, >, [, ] characters
    cleaned = str(text).replace("<", "").replace(">", "").replace("[", "").replace("]", "")

    # Tokenize the cleaned text
    words = cleaned.strip().split()

    # Count disfluencies (words matching DISFLUENCY_TAGS after cleaning)
    disfluencies = sum(1 for word in words if word.lower() in [tag.lower() for tag in DISFLUENCY_TAGS])

    # Return the density, handling division by zero
    return disfluencies / len(words) if words else 0.0

# Apply the new disfluency density calculation to the existing df
df['disfluency_density'] = df['unsanitized_normalized'].apply(calculate_disfluency_density_user_logic)

print("Calculated 'disfluency_density' column using the specified logic.")
display(df[['verbatim', 'unsanitized_normalized', 'disfluency_density']].head())

# Note: This new logic does not require the temporary columns from the previous disfluency calculation.
# The 'total_words_verbatim' drop was also removed as it was not defined in this cell.


Calculated 'disfluency_density' column using the specified logic.


,verbatim,unsanitized_normalized,disfluency_density
0,ఈ వార్తాదారం నుండి తాజా ముఖ్యాంశాలు ఏంటి,@noise-start ఈ వార్తా ధార నుండి తాజా ముఖ్యాంశా...,0.0
1,నేను నాలుగు ఎనర్జాయర్స్ ఆల్కలీన్ బ్యాటరీలు ఆర్...,నేను నాలుగు ఎనర్జాయర్స్ [energiors] ఆల్కలీన్ [...,0.0
2,ఆమె గతంలో పంతొమ్మిదొందల టొంభై ఏడులో మిస్ ఇంటర్...,@noise-start ఆమె గతంలో పంతొమ్మిది వందల తొంభై ఏ...,0.0
3,ఇంపాల్లో వాతావరణం ఎలా ఉంది,ఇంపాల్‌లో వాతావరణం ఎలా ఉంది,0.0
4,సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ఉందా,@noise-start సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ...,0.0


##Character Error Rate

## CER for Accents

$$
CER = \frac{S + D + I}{N}
$$

Where:
- S = Number of substitutions  
- D = Number of deletions  
- I = Number of insertions  
- N = Total number of characters  

CER is used to quantify accent variations or mispronunciations by comparing
the **verbatim** transcript with the **normalized** transcript.


In [20]:
import jiwer
df = df.dropna(subset=['verbatim', 'normalized'])

# Define WER calculation: normalized = reference, verbatim = hypothesis
def calculate_cer(row):
    ref = str(row['normalized'])
    hyp = str(row['verbatim'])
    try:
        return jiwer.cer(ref, hyp)
    except:
        return None

# Apply WER row-wise
df['cer'] = df.apply(calculate_cer, axis=1)
display(df.head())


,text,duration,lang,samples,verbatim,normalized,speaker_id,scenario,task_name,gender,...,verification_report,unsanitized_verbatim,unsanitized_normalized,file,segment,audio,batch,rsml,code_mix_density,cer
0,ఈ వార్తా ధార నుండి తాజా ముఖ్యాంశాలు ఏంటి,5.383,te,86128,ఈ వార్తాదారం నుండి తాజా ముఖ్యాంశాలు ఏంటి,ఈ వార్తా ధార నుండి తాజా ముఖ్యాంశాలు ఏంటి,S4258217800311013,Read,Alexa Commands,Female,...,"{'sst': False, 'comments': '', 'decision': 'ex...",@noise-start ఈ వార్తాదారం నుండి తాజా ముఖ్యాంశా...,@noise-start ఈ వార్తా ధార నుండి తాజా ముఖ్యాంశా...,1,1,te-1-1-1.wav,1,@noise-start ఈ <వార్తాదారం>(వార్తాధార) నుండి త...,0.000000,0.075
1,నేను నాలుగు ఎనర్జాయర్స్ ఆల్కలీన్ బ్యాటరీలు ఆర్...,8.989,te,143824,నేను నాలుగు ఎనర్జాయర్స్ ఆల్కలీన్ బ్యాటరీలు ఆర్...,నేను నాలుగు ఎనర్జాయర్స్ ఆల్కలీన్ బ్యాటరీలు ఆర్...,S4258785200319518,Read,Bigbasket Commands,Male,...,"{'decision': 'excellent', 'low_volume': False,...",నేను నాలుగు ఎనర్జాయర్స్ ఆల్కలీన్ బ్యాటరీలు ఆర్...,నేను నాలుగు ఎనర్జాయర్స్ [energiors] ఆల్కలీన్ [...,2,1,te-1-2-1.wav,1,నేను #NUMBER{నాలుగు}(4) #PRODUCT{ఎనర్జాయర్స్}(...,0.266667,0.000
2,ఆమె గతంలో పంతొమ్మిది వందల తొంభై ఏడులో మిస్ ఇంట...,7.516,te,120256,ఆమె గతంలో పంతొమ్మిదొందల టొంభై ఏడులో మిస్ ఇంటర్...,ఆమె గతంలో పంతొమ్మిది వందల తొంభై ఏడులో మిస్ ఇంట...,S4257895000391352,Read,Wikipedia Sentences,Female,...,"{'decision': 'excellent', 'low_volume': False,...",@noise-start ఆమె గతంలో పంతొమ్మిదొందల టొంభై ఏడు...,@noise-start ఆమె గతంలో పంతొమ్మిది వందల తొంభై ఏ...,3,1,te-1-3-1.wav,1,NaN,0.214286,0.050
3,ఇంపాల్లో వాతావరణం ఎలా ఉంది,2.550,te,40800,ఇంపాల్లో వాతావరణం ఎలా ఉంది,ఇంపాల్లో వాతావరణం ఎలా ఉంది,S4259096100371020,Read,Alexa Commands,Female,...,"{'sst': False, 'comments': '', 'decision': 'ex...",ఇంపాల్‌లో వాతావరణం ఎలా ఉంది,ఇంపాల్‌లో వాతావరణం ఎలా ఉంది,4,1,te-1-4-1.wav,1,NaN,0.000000,0.000
4,సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ఉందా,3.798,te,60768,సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ఉందా,సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ఉందా,S4258012200308577,Read,Bigbasket Commands,Male,...,"{'decision': 'excellent', 'low_volume': False,...",@noise-start సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ...,@noise-start సుగంధ ద్రవ్యాలపై ఏదైనా పండగ ఆఫర్ ...,5,1,te-1-5-1.wav,1,NaN,0.142857,0.000


## Top Code-Mixed Density Segments

### Subtask:
Identify and display the top 10 segments with the highest `code_mix_density`.


**Reasoning**:
To identify the top 10 segments with the highest code-mix density, I need to sort the DataFrame by the 'code_mix_density' column in descending order and then select the first 10 rows, displaying the specified columns.



In [43]:
top_code_mix = df.sort_values(by='code_mix_density', ascending=False).head(10)
display(top_code_mix[['text', 'verbatim', 'normalized', 'code_mix_density']])


,text,verbatim,normalized,code_mix_density
190351,ఓకే,ఓకే,ఓకే,1.000000
193069,హలో,హలో,హలో,1.000000
57575,ఓకే,ఓకే,ఓకే,0.666667
193076,ఓకే మేడం,ఒకే మేడం,ఓకే మేడం,0.666667
76248,ఓకే,ఓకే,ఓకే,0.666667
57789,ఓకే మేడం,ఓకే మేడం,ఓకే మేడం,0.600000
153486,ఇండియా పాకిస్థాన్ కెనడా జపాన్ చైనా,ఇండియా పాకిస్థాన్ కెనడా జపాన్ చైనా,ఇండియా పాకిస్థాన్ కెనడా జపాన్ చైనా,0.555556
203279,ఓకే సార్ త్యాంక్ యూ,ఓకే సార్ త్యాంక్ యూ,ఓకే సార్ త్యాంక్ యూ,0.500000
7240,ఓకే మేడం,ఓకే మేడం,ఓకే మేడం,0.500000
203301,మొబైలు,మొబైలు,మొబైలు,0.500000


## Top Disfluency Density Segments

### Subtask:
Identify and display the top 10 segments with the highest `disfluency_density`.

#### Instructions
1. Sort the DataFrame `df` by the 'disfluency_density' column in descending order.
2. Select the first 10 rows of the sorted DataFrame.
3. Display the 'text', 'verbatim', 'normalized', and 'disfluency_density' columns for these top 10 segments.

**Reasoning**:
To identify the top 10 segments with the highest disfluency density, I need to sort the DataFrame by the 'disfluency_density' column in descending order and then select the first 10 rows, displaying the specified columns.



In [44]:
top_disfluency = df.sort_values(by='disfluency_density', ascending=False).head(10)
display(top_disfluency[['text', 'verbatim', 'normalized', 'disfluency_density']])

,text,verbatim,normalized,disfluency_density
46003,అవునండి,అవునండి,అవునండి,0.500000
6948,ట్వంటీ పెట్టుకుందాం,ట్వంటీ పెట్టుకుందాం,ట్వంటీ పెట్టుకుందాం,0.333333
212160,అవును చెప్పండి,అవును చెప్పండి,అవును చెప్పండి,0.200000
38257,రేపు వచ్చేసేయండి,రేపు వచ్చేసేయండి,రేపు వచ్చేసేయండి,0.200000
147105,బొంగు చికెను,బొంగు చికెను,బొంగు చికెను,0.166667
61821,మరి మీ అదే బుక్కులు వేరే,మరి మీ అదే బుక్కులు వేరే,మరి మీ అదే బుక్కులు వేరే,0.166667
12456,అది కేజీ వచ్చి ఎంతండి,అది కేజీ వచ్చి ఎంతండి,అది కేజీ వచ్చి ఎంతండి,0.125000
145181,వంట వంట చేసుకోవడానికి ఉపయోగించే రైస్ కుక్కర్,వంట వంట చేసుకోవడానికి ఉపయోగించే రైస్ కుక్కరు,వంట వంట చేసుకోవడానికి ఉపయోగించే రైస్ కుక్కర్,0.111111
11150,ఎంత పట్టచ్చు అండి ఎంత అవుతదండి ఇప్పుడు ఇంచుమించు,ఎంత పట్టొచ్చండి ఎంత అవుతదండి ఇప్పుడు ఇంచుమించు,ఎంత పట్టచ్చు అండి ఎంత అవుతదండి ఇప్పుడు ఇంచుమించు,0.100000
146430,అందరు ఫార్మర్స్ ఉంటారు మా ఇంటి చుట్టు రైతులు,అందరూ ఫార్మర్స్ ఉంటారు మా ఇంటి చుట్టూ రైతులు,అందరు ఫార్మర్స్ ఉంటారు మా ఇంటి చుట్టు రైతులు,0.100000


## Top Character Error Rate (CER) Segments

### Subtask:
Identify and display the top 10 segments with the highest `cer`.

#### Instructions
1. Sort the DataFrame `df` by the 'cer' column in descending order.
2. Select the first 10 rows of the sorted DataFrame.
3. Display the 'text', 'verbatim', 'normalized', and 'cer' columns for these top 10 segments.

In [45]:
top_cer = df.sort_values(by='cer', ascending=False).head(10)
display(top_cer[['text', 'verbatim', 'normalized', 'cer']])

,text,verbatim,normalized,cer
195703,హలో,హలో తెలిస్,హలో,2.333333
138084,ఎట్లా అంటే ఇక పోయే ముం,ఎట్లా అంటే ఇక పోయే ముందు ఇక కొబ్బరికాయలు కొడతర...,ఎట్లా అంటే ఇక పోయే ముం,2.318182
50044,ఆ,ఆయ్,ఆ,2.000000
13301,ఓ,ఓక్,ఓ,2.000000
215851,ఆ,ఆయ్,ఆ,2.000000
50045,ఆ,ఆయ్,ఆ,2.000000
37457,ఆ,ఆయి,ఆ,2.000000
194228,ఆ డబ్బులున్న వారైతే పట్టణముకి వెళ్తారు ఆసుపత్ర...,ఆ డబ్బులున్న వారైతే పట్టణముకి వెళ్తారు ఆసుపత్ర...,ఆ డబ్బులున్న వారైతే పట్టణముకి వెళ్తారు ఆసుపత్ర...,1.590909
216646,ఓకే,ఓకే ఓకే,ఓకే,1.333333
214527,మేడం,మ్యాడమ్,మేడం,1.250000


## Normalize Metrics

### Subtask
Normalize `code_mix_density`, `disfluency_density`, and `cer` to a 0–1 scale.

### Instructions

1. For each metric  
   (`code_mix_density`, `disfluency_density`, `cer`), compute the minimum and
   maximum values across the entire DataFrame.

2. Apply **min–max normalization** to each metric using the formula:

   $$
   X_{\text{normalized}} = \frac{X - \min(X)}{\max(X) - \min(X)}
   $$

3. Handle the edge case where all values of a metric are identical:

   $$
   \text{If } \max(X) - \min(X) = 0,\quad X_{\text{normalized}} = 0
   $$

   (Alternatively, a constant value such as 0.5 may be used to indicate no variation.)

4. Create new columns in the DataFrame for the normalized metrics:
   - `code_mix_density_normalized`
   - `disfluency_density_normalized`
   - `cer_normalized`

5. Display the first few rows of the DataFrame to verify the normalization.

   ```python
   df.head()


In [46]:
columns_to_normalize = ['code_mix_density', 'disfluency_density', 'cer']

for col in columns_to_normalize:
    min_val = df[col].min()
    max_val = df[col].max()

    if max_val - min_val == 0:
        # If all values are the same, set normalized value to 0.5 (or 0 depending on desired behavior)
        df[col + '_normalized'] = 0.5
    else:
        df[col + '_normalized'] = (df[col] - min_val) / (max_val - min_val)

print("DataFrame with normalized columns added.")
display(df[['code_mix_density', 'code_mix_density_normalized',
            'disfluency_density', 'disfluency_density_normalized',
            'cer', 'cer_normalized']].head())

DataFrame with normalized columns added.


,code_mix_density,code_mix_density_normalized,disfluency_density,disfluency_density_normalized,cer,cer_normalized
0,0.000000,0.000000,0.0,0.0,0.075,0.032143
1,0.266667,0.266667,0.0,0.0,0.000,0.000000
2,0.214286,0.214286,0.0,0.0,0.050,0.021429
3,0.000000,0.000000,0.0,0.0,0.000,0.000000
4,0.142857,0.142857,0.0,0.0,0.000,0.000000


## Calculate Combined High Score


Create a new column that represents a composite score by summing the normalized values of `code_mix_density`, `disfluency_density`, and `cer`.


In [47]:
df['combined_score'] = df['code_mix_density_normalized'] + df['disfluency_density_normalized'] + df['cer_normalized']

print("DataFrame with 'combined_score' column calculated and added.")
display(df[['code_mix_density_normalized', 'disfluency_density_normalized', 'cer_normalized', 'combined_score']].head())

DataFrame with 'combined_score' column calculated and added.


,code_mix_density_normalized,disfluency_density_normalized,cer_normalized,combined_score
0,0.000000,0.0,0.032143,0.032143
1,0.266667,0.0,0.000000,0.266667
2,0.214286,0.0,0.021429,0.235714
3,0.000000,0.0,0.000000,0.000000
4,0.142857,0.0,0.000000,0.142857


In [50]:
top_combined_score = df.sort_values(by='combined_score', ascending=False).head(30)
display(top_combined_score[['batch','file', 'segment', 'text', 'verbatim', 'normalized', 'code_mix_density_normalized', 'disfluency_density_normalized', 'cer_normalized', 'combined_score']])

,batch,file,segment,text,verbatim,normalized,code_mix_density_normalized,disfluency_density_normalized,cer_normalized,combined_score
195703,46,1018,2,హలో,హలో తెలిస్,హలో,0.5,0.0,1.000000,1.500000
216646,8,922,9,ఓకే,ఓకే ఓకే,ఓకే,0.5,0.0,0.571429,1.071429
75005,23,615,25,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
43035,18,18,10,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
187498,45,255,10,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
74988,23,615,8,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
75002,23,615,22,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
214527,8,440,22,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
73317,23,382,3,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
74990,23,615,10,మేడం,మ్యాడమ్,మేడం,0.5,0.0,0.535714,1.035714
